# Getting Started with Stochastic Engine

This notebook introduces the core features of the Stochastic Engine library.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import stochastic engine
from stochastic_engine import GBM, OrnsteinUhlenbeck, BlackScholes, VaR

## 1. Simulating Stock Prices with GBM

Geometric Brownian Motion is the standard model for stock prices.

In [ ]:
# Create a GBM process
gbm = GBM(
    S0=100,     # Initial stock price
    mu=0.08,    # 8% annual drift (expected return)
    sigma=0.2,  # 20% annual volatility
    seed=42     # For reproducibility
)

# Simulate 5 paths over 1 year
paths = gbm.simulate(T=1, steps=252, n_paths=5)

# Plot the paths
plt.figure(figsize=(10, 6))
t = np.linspace(0, 1, 253)
for i in range(5):
    plt.plot(t, paths[i], alpha=0.7)
plt.xlabel('Time (years)')
plt.ylabel('Stock Price ($)')
plt.title('Simulated Stock Price Paths (GBM)')
plt.grid(True, alpha=0.3)
plt.show()

## 2. Terminal Distribution

Let's simulate many paths and look at the distribution of final prices.

In [ ]:
# Simulate 10,000 paths
gbm.reset_rng(42)
terminal_prices = gbm.sample(t=1, n_samples=10000)

# Plot histogram
plt.figure(figsize=(10, 6))
plt.hist(terminal_prices, bins=50, density=True, alpha=0.7, edgecolor='black')
plt.axvline(gbm.mean(1), color='red', linestyle='--', label=f'Expected: ${gbm.mean(1):.2f}')
plt.axvline(terminal_prices.mean(), color='green', linestyle=':', label=f'Sample Mean: ${terminal_prices.mean():.2f}')
plt.xlabel('Terminal Stock Price ($)')
plt.ylabel('Density')
plt.title('Distribution of Terminal Stock Prices')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Theoretical Mean: ${gbm.mean(1):.2f}")
print(f"Sample Mean: ${terminal_prices.mean():.2f}")
print(f"Theoretical Std: ${gbm.std(1):.2f}")
print(f"Sample Std: ${terminal_prices.std():.2f}")

## 3. Option Pricing with Black-Scholes

Price European options and calculate Greeks.

In [ ]:
# Price a call option
call = BlackScholes(
    S=100,      # Current stock price
    K=105,      # Strike price
    T=0.5,      # 6 months to expiration
    r=0.05,     # 5% risk-free rate
    sigma=0.2,  # 20% volatility
    option_type='call'
)

print("Call Option:")
print(f"  Price: ${call.price:.4f}")
print(f"  Delta: {call.delta:.4f}")
print(f"  Gamma: {call.gamma:.4f}")
print(f"  Vega:  {call.vega:.4f} (per 1% vol change)")
print(f"  Theta: {call.theta:.4f} (daily decay)")
print(f"  Rho:   {call.rho:.4f} (per 1% rate change)")

In [ ]:
# Plot delta across different stock prices
stock_prices = np.linspace(80, 120, 100)
bs = BlackScholes(S=stock_prices, K=100, T=0.5, r=0.05, sigma=0.2)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(stock_prices, bs.price)
axes[0].set_xlabel('Stock Price ($)')
axes[0].set_ylabel('Option Price ($)')
axes[0].set_title('Call Option Price')
axes[0].grid(True, alpha=0.3)

axes[1].plot(stock_prices, bs.delta)
axes[1].set_xlabel('Stock Price ($)')
axes[1].set_ylabel('Delta')
axes[1].set_title('Delta (Sensitivity to Stock Price)')
axes[1].grid(True, alpha=0.3)

axes[2].plot(stock_prices, bs.gamma)
axes[2].set_xlabel('Stock Price ($)')
axes[2].set_ylabel('Gamma')
axes[2].set_title('Gamma (Rate of Change of Delta)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Mean-Reverting Processes

The Ornstein-Uhlenbeck process is useful for modeling interest rates and spreads.

In [ ]:
# Create an OU process for interest rates
ou = OrnsteinUhlenbeck(
    X0=0.08,     # Starting at 8%
    mu=0.05,     # Long-term mean of 5%
    theta=0.5,   # Mean reversion speed
    sigma=0.01,  # Volatility
    seed=42
)

print(f"Half-life of mean reversion: {ou.half_life:.2f} years")

# Simulate paths
paths = ou.simulate(T=10, steps=2520, n_paths=5)

plt.figure(figsize=(10, 6))
t = np.linspace(0, 10, 2521)
for i in range(5):
    plt.plot(t, paths[i] * 100, alpha=0.7)  # Convert to percentage
plt.axhline(ou.mu * 100, color='red', linestyle='--', label=f'Long-term mean: {ou.mu*100:.1f}%')
plt.xlabel('Time (years)')
plt.ylabel('Interest Rate (%)')
plt.title('Simulated Interest Rate Paths (Ornstein-Uhlenbeck)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Risk Metrics

Calculate Value at Risk and other risk measures.

In [ ]:
# Generate sample returns
np.random.seed(42)
returns = np.random.normal(0.0005, 0.02, 252)  # Daily returns for 1 year

# Calculate VaR
var = VaR(returns, confidence=0.95)

print("Risk Metrics (95% confidence):")
print(f"  Historical VaR:  {var.historical():.2%}")
print(f"  Parametric VaR:  {var.parametric():.2%}")
print(f"  10-day VaR:      {var.scale_to_horizon(var.historical(), 10):.2%}")

In [ ]:
# Visualize VaR
plt.figure(figsize=(10, 6))
plt.hist(returns, bins=50, density=True, alpha=0.7, edgecolor='black')
var_95 = -var.historical()  # Negative because we want the left tail
plt.axvline(var_95, color='red', linestyle='--', linewidth=2, label=f'95% VaR: {-var_95:.2%}')
plt.fill_betweenx([0, 25], returns.min(), var_95, alpha=0.3, color='red')
plt.xlabel('Daily Return')
plt.ylabel('Density')
plt.title('Return Distribution with VaR')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Summary

In this notebook, we covered:

1. **GBM Simulation** - Model stock prices with geometric Brownian motion
2. **Black-Scholes** - Price European options and calculate Greeks
3. **Ornstein-Uhlenbeck** - Model mean-reverting processes like interest rates
4. **VaR** - Measure portfolio risk

For more advanced topics, see the other notebooks in this series.